## Notebook for the Smillie, 2019 QC 
### Developed by: Anna Maguza

### Institute of Computational Biology - Computational Health Centre - Hemlholtz Munich

### 16th June 2023

#### Load required packages

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as an
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#### Setup Cells

In [3]:
%matplotlib inline

In [4]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

/Users/anna.maguza/miniconda3/envs/scanpy_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.2.2 statsmodels==0.13.5 pynndescent==0.5.8


#### Upload Data

In [5]:
input = '/Users/anna.maguza/Desktop/Data/Gut_project/Smillie/Anndata_raw/Smillie_ulcerative_colitis_anndata.h5ad'
adata = sc.read_h5ad(input)

In [6]:
adata

AnnData object with n_obs × n_vars = 240373 × 21784
    obs: 'Location', 'Donor_ID', 'Sample_ID', 'CellType', 'n_counts', 'n_genes', 'Diagnosis'